In [11]:
from pyspark.sql import SQLContext
from pyspark.context import SparkContext
from pyspark.conf import SparkConf

conf = SparkConf().setAppName("App")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '4G')
        .set('spark.driver.memory', '4G')
        .set('spark.driver.maxResultSize', '3G'))

sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)

In [12]:
datapath  = '../../data'
neo4jpath = '../../neo4j/spark' 

In [13]:
def load(datapath):
    processed_path = '{}{}'.format(datapath,'/urbs/processed/')

    sqlContext.read.parquet(processed_path+'linhas/').registerTempTable("linhas")
    sqlContext.read.parquet(processed_path+'pontoslinha/').registerTempTable("pontos_linha")

    sqlContext.read.parquet(processed_path+'tabelaveiculo/').registerTempTable("tabela_veiculo")

def save(query, target_path):
    sqlContext.sql(query).coalesce(1) \
        .write.mode('overwrite')      \
        .option("header", "true")     \
        .format("csv")                \
        .save(target_path)


def show(query, n=10):
    sqlContext.sql(query).show(n)


def run(query):
    sqlContext.sql(query)

load(datapath)

In [14]:
query = """
select cod
      ,categoria_servico
      ,nome
      ,nome_cor
      ,somente_cartao
    from linhas
"""
show(query)

target_path = '{}{}'.format(neo4jpath,'/lines/')
save(query, target_path=target_path)

+---+-----------------+--------------------+--------+--------------+
|cod|categoria_servico|                nome|nome_cor|somente_cartao|
+---+-----------------+--------------------+--------+--------------+
|464|     CONVENCIONAL|A. MUNHOZ / J. BO...| AMARELA|             S|
|226|      ALIMENTADOR|              ABAETE| LARANJA|             N|
|182|          TRONCAL|           ABRANCHES| AMARELA|             N|
|332|      ALIMENTADOR|            ACROPOLE| LARANJA|             N|
|334|      ALIMENTADOR|            AGRICOLA| LARANJA|             N|
|265|     CONVENCIONAL|   AHU / LOS ANGELES| AMARELA|             S|
|560|     CONVENCIONAL|        ALFERES POLI| AMARELA|             S|
|232|      ALIMENTADOR|             ALIANCA| LARANJA|             N|
|629|      ALIMENTADOR|      ALTO BOQUEIR?O| LARANJA|             N|
|373|          TRONCAL|         ALTO TARUM?| AMARELA|             N|
+---+-----------------+--------------------+--------+--------------+
only showing top 10 rows



In [15]:
query = """
select distinct
       nome
      ,num
      ,tipo
      ,lat
      ,lon
      from pontos_linha
"""
show(query)

target_path = '{}{}'.format(neo4jpath,'/busstops/')
save(query, target_path=target_path)

+--------------------+------+---------------+----------------+----------------+
|                nome|   num|           tipo|             lat|             lon|
+--------------------+------+---------------+----------------+----------------+
|Rua Mal. Otavio S...|170052|Novo mobiliario|-25.522075360032|-49.297146011886|
|Rua Coronel Amazo...|120957|Novo mobiliario| -25.40002279702|-49.256587232155|
|Rua Epaminondas S...|130376|          Domus| -25.42217200427|-49.199223847336|
|Avenida Candido H...|190178|  Chapeu chines|-25.427784844752| -49.29681682209|
|Rua Engenheiro Jo...|170956|  Chapeu chines|-25.570770103902|-49.331832261032|
|Av. Juscelino Kub...|180189|          Domus|       -25.46951|       -49.35415|
|Rua Angelo Massig...|190055| Sem demarcac?o|-25.416395762467|-49.354267347221|
|Rua Cel. Francisc...|140177|Novo mobiliario|-25.465791881341|-49.234808483467|
|Rua Jose Benedito...|180279|  Chapeu chines|-25.462834314069|-49.329030202265|
|Estac?o Tubo Silv...|108037|   Estac?o 

In [16]:
query_view_rota_sequenciada = """
CREATE OR REPLACE TEMPORARY VIEW rota_sequenciada AS
   select 	pseq.cod_linha
           ,pseq.sentido_linha
           ,pseq.seq_inicio
           ,pseq.seq_fim
           ,pseq.ponto_inicio
           ,pseq.nome_ponto_inicio
           ,pseq.ponto_final
           ,pseq.nome_ponto_final
           ,li.CATEGORIA_SERVICO as categoria_servico
           ,li.NOME as nome_linha
           ,li.NOME_COR as nome_cor
           ,li.SOMENTE_CARTAO as somente_cartao
           ,pseq.year
           ,pseq.month
           ,pseq.day
                 from (select
                               p1.COD as cod_linha
                              ,p1.SENTIDO  as sentido_linha
                              ,p1.SEQ      as seq_inicio
                              ,p2.SEQ      as seq_fim
                              ,p1.NUM      as ponto_inicio
                              ,p1.NOME     as nome_ponto_inicio
                              ,p2.NUM      as ponto_final
                              ,p2.NOME     as nome_ponto_final
                              ,p1.year
                              ,p1.month
                              ,p1.day
                              from pontos_linha P1
                              inner join pontos_linha p2
                                  on (p1.SEQ+1 = p2.SEQ
                                       and p1.COD = p2.COD
                                       and p1.SENTIDO = p2.SENTIDO
                                       and p1.year = p2.year
                                       and p1.month=p2.month and p1.day=p2.day)
                              ) pseq
                              inner join linhas li
                                     on (pseq.cod_linha = li.COD
                                     and pseq.year = li.year
                                     and pseq.month=li.month
                                     and pseq.day=li.day
                                     )
                              order by pseq.cod_linha
                                      ,pseq.sentido_linha
                                      ,pseq.seq_inicio
                                      ,pseq.seq_fim
"""

run(query_view_rota_sequenciada)

In [17]:
query_rota_sequenciada = """
    select distinct
           cod_linha
          ,sentido_linha
          ,seq_inicio
          ,seq_fim
          ,ponto_inicio
          ,nome_ponto_inicio
          ,ponto_final
          ,nome_ponto_final
          ,categoria_servico
          ,nome_linha
          ,nome_cor
          ,somente_cartao
         from rota_sequenciada
         -- where cod_linha = '010' and sentido_linha = 'Horario'
         order by int(seq_inicio)
                 ,int(seq_fim)
"""
show(query_rota_sequenciada,n=10)

target_path = '{}{}'.format(neo4jpath,'/routes/')

save(query_rota_sequenciada, target_path=target_path)

+---------+---------------+----------+-------+------------+--------------------+-----------+--------------------+-----------------+--------------------+--------+--------------+
|cod_linha|  sentido_linha|seq_inicio|seq_fim|ponto_inicio|   nome_ponto_inicio|ponto_final|    nome_ponto_final|categoria_servico|          nome_linha|nome_cor|somente_cartao|
+---------+---------------+----------+-------+------------+--------------------+-----------+--------------------+-----------------+--------------------+--------+--------------+
|      002|   Anti-Horario|         1|      2|      103003|Praca 19 de Dezem...|     110133|Rua Trajano Reis,...|  CIRCULAR CENTRO|CIRCULAR CENTRO (...|  BRANCA|             S|
|      010|        Horario|         1|      2|      190176|R. Tapajos, 948- ...|     120110|R. Carlos Pioli, ...|     INTERBAIRROS|INTERBAIRROS I (H...|   VERDE|             N|
|      010|        Horario|         1|      2|      140187|Rua Iapo, 842 - P...|     120110|R. Carlos Pioli, ...|  

In [18]:
query = """
with start_end as (
    select  cod
           ,sentido
           ,min(int(seq)) as start_trip
           ,max(int(seq)) as end_trip
         from pontos_linha
    group by cod,sentido
)
 select ps.cod     as line_code
       ,ps.sentido
       ,ps.num     as origin
       ,ps.nome    as ponto_origem
       ,pe.num     as destination
       ,pe.nome    as ponto_destino
  from start_end  ss
     inner join pontos_linha ps on (ps.cod = ss.cod  and ps.sentido = ss.sentido and ps.seq = ss.start_trip)
     inner join pontos_linha pe on (pe.cod = ss.cod  and pe.sentido = ss.sentido and pe.seq = ss.end_trip)
     -- where ps.cod = '666'
"""

target_path = '{}{}'.format(neo4jpath,'/trip-endpoints/')

save(query, target_path=target_path)

In [19]:
query = """
select cod_linha
      ,cod_ponto
      ,horario
      ,nome_linha
      ,tabela
      ,veiculo
      from tabela_veiculo
      order by cod_linha,horario
"""

target_path = '{}{}'.format(neo4jpath,'/schedules/')
save(query, target_path=target_path)

In [20]:
query = """
with query_1 as (
select cod_linha     as line_code
      ,cod_ponto     as start_point
      ,horario       as start_time
      ,tabela        as time_table
      ,veiculo       as vehicle
      ,lead(horario) over(partition by cod_linha,tabela,veiculo order by cod_linha, horario)   as end_time
      ,lead(cod_ponto) over(partition by cod_linha,tabela,veiculo order by cod_linha, horario) as end_point
      from tabela_veiculo
      order by cod_linha,horario
)
select * from query_1
"""

target_path = '{}{}'.format(neo4jpath,'/trips/')

save(query, target_path=target_path)